In [1]:
data_samples <- read.csv("data_melatonion.csv", check.names = TRUE)
clock_stool <- read.csv("Clock_stool.csv", check.names = TRUE)

dim(data_samples)

[1] 712   6

In [2]:

Study_variable <- 'Melatonin_pg_g_Stool'  
as.symbol(Study_variable)
Ages <- c('all' , '3 months', '6 months', '12 months')

Age <- Ages[1]     #'all' , '3 months', '6 months';, '12 months'


data_act <- read.csv("BISQ_samples5.csv")
#data_act <- read.csv("Act2.csv")
#head(data_act)
dim(data_act)
head(data_act)

Melatonin_pg_g_Stool

[1] 632  17

Cohort SubjectID Timepoint Sample_Code_16s Sleep_duration_night
1 SDEGU  S001      3         S001-03         9                   
2 SDEGU  S001      6         S001-06         10                  
3 SDEGU  S001      12        S001-12         11.5                
4 SDEGU  S003      3         S003-03         9                   
5 SDEGU  S003      6         S003-06         9.75                
6 SDEGU  S003      12        S003-12         10                  
  Sleep_duration_day Awake_night Sleep.Latency Bedtime 
1 nan                1           0.25          23:10   
2 2.5                1.5         0             21:00:00
3 2                  0.33        0.08          19:00   
4 3.5                1           0.25          20:30   
5 nan                10          0.25          nan     
6 35                 0.25        0.16          20:30   
  Sleep_regularly_weekdays Sleep_regularly_weekend Wake_UP Duration_awake
1 2                        2                       2       1             
2 2                        2                       2       5             
3 1                        1                       2       1             
4 2                        2                       2       3             
5 2                        2                       2       5             
6 2                        2                       2       1             
  Daytime_nap Daytime_nap_frequency Duration_aytime_nap Daytime_nap_night
1 3           7                     4                   1                
2 3           7                     2                   1                
3 2           7                     3                   1                
4 3           7                     3                   1                
5 1           7                     6                   1                
6 1           7                     5                   1

In [3]:
colnames(data_act)

[1] "Cohort"                   "SubjectID"               
 [3] "Timepoint"                "Sample_Code_16s"         
 [5] "Sleep_duration_night"     "Sleep_duration_day"      
 [7] "Awake_night"              "Sleep.Latency"           
 [9] "Bedtime"                  "Sleep_regularly_weekdays"
[11] "Sleep_regularly_weekend"  "Wake_UP"                 
[13] "Duration_awake"           "Daytime_nap"             
[15] "Daytime_nap_frequency"    "Duration_aytime_nap"     
[17] "Daytime_nap_night"

In [4]:

#data_act <- data_act[, c('Sample_Code_16s', 'Sleep_Latency', 'Longest_wake_night', 'WASO', 'Sleep_Efficiency')]
# Assigning new column names to an existing dataframe
#colnames(data_act) <- c('Sample_Code_16s', 'Sleep.Latency', 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night')
#data_act

In [5]:
act_vars <- c('Sleep.Latency' , 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool')
act_var <- act_vars[4]


In [6]:
# Load necessary library
library(dplyr)

# Define the preprocessing function
convert_time_to_numeric <- function(df, act_var) {
  
  # Check each value in the column: if it's not numeric and contains ":", convert it
  df <- df %>%
    mutate(!!act_var := ifelse(grepl(":", as.character(!!sym(act_var))),
                               sapply(strsplit(as.character(!!sym(act_var)), ":"), 
                                      function(x) as.numeric(x[1]) + as.numeric(x[2]) / 60),
                               as.numeric(!!sym(act_var))))  # Keep numeric values as they are
  
  cat(paste(act_var, "has been checked and converted where needed.\n"))
  
  return(df)
}

# Example usage:
data_act <- convert_time_to_numeric(data_act, 'Sleep.Latency')
clock_stool <- convert_time_to_numeric(clock_stool, 'Clock_stool')



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Sleep.Latency = ifelse(...)`.
Caused by warning in `ifelse()`:
! NAs introduced by coercion"


Sleep.Latency has been checked and converted where needed.


Warning message:
"There were 2 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Clock_stool = ifelse(...)`.
Caused by warning in `FUN()`:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."


Clock_stool has been checked and converted where needed.


In [7]:
# Function to extract age group based on conditions
 
extract_age_group <- function(sample_id) {

      # Check if the sample ID is NA
  if (is.na(sample_id)) {
    return(NA)
  }

    
  # Check if the sample ID starts with 'S'
  if (startsWith(sample_id, 'S')) {
    # Split the sample ID by '-'
    parts <- strsplit(sample_id, "-")[[1]]
    # Get the second division
    division <- parts[2]
    # Extract the first two characters
    return(as.integer(substr(division, 1, 2)))
  }
    
  # Split the sample ID by '-'
  parts <- strsplit(sample_id, "-")[[1]]
  # Get the second division
  division <- parts[2]
  # Extract the last character
  last_char <- substring(division, nchar(division), nchar(division))
  # Check if the sample ID starts with 'I'
  if (startsWith(sample_id, 'I')) {
    # If so, multiply the last character by 3
    return(as.integer(last_char) * 3)
    #return(0)       # Sleepy cohort ages between 5 and 31 months, so, will be excluded
  } else {
    # Otherwise, return the last character as integer
    return(as.integer(last_char))
  }
}

# Apply the function to create the age group column
data_samples$age_group <- sapply(data_samples$Sample_Code_16s, extract_age_group)

# View the updated data
#print(data)


dim(data_samples)
data_samples[1:3,]

[1] 712   7

Cohort Sample_id Timepoint Day Sample_Code_16s Melatonin_pg_g_Stool age_group
1 SPIN   M001      t2        1   M001-M6         145.53               6        
2 SPIN   M001      t2        2   M001-06         176.03               6        
3 SPIN   M002      t2        1   M002-M6         275.78               6

In [8]:
data_act <- merge(data_act,clock_stool, by = "Sample_Code_16s")


In [9]:
# Load necessary libraries
library(dplyr)
library(rlang)

# Define a function to clean the data and remove outliers
remove_outliers <- function(df, column_name) {
  
  # Convert the column name to a symbol
  column_sym <- sym(column_name)
  
  # Clean the data by ensuring the specified column is non-NaN and numeric
  cleaned_data <- df %>%
    filter(!is.na(!!column_sym)) %>%
    mutate(!!column_sym := as.numeric(!!column_sym))
  
  # Calculate Q1 (25th percentile) and Q3 (75th percentile)
  Q1 <- quantile(cleaned_data[[column_name]], 0.15, na.rm = TRUE)
  Q3 <- quantile(cleaned_data[[column_name]], 0.85, na.rm = TRUE)
  
  # Calculate the IQR (Interquartile Range)
  IQR_value <- IQR(cleaned_data[[column_name]], na.rm = TRUE)
  
  # Define lower and upper bounds
  lower_bound <- Q1 - 1.5 * IQR_value
  upper_bound <- Q3 + 1.5 * IQR_value
  
  # Remove rows that are considered outliers
  cleaned_data <- cleaned_data %>%
    filter((.data[[column_name]] >= lower_bound) & (.data[[column_name]] <= upper_bound))
  
  return(cleaned_data)
}

# Example usage:
cleaned_data_act <- remove_outliers(data_act, act_var)
cleaned_data_samples <- remove_outliers(data_samples, Study_variable)
# head(cleaned_samples)
dim(cleaned_data_act)
dim(cleaned_data_samples)


Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Awake_night = as.numeric(Awake_night)`.
Caused by warning:
! NAs introduced by coercion"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `Melatonin_pg_g_Stool = as.numeric(Melatonin_pg_g_Stool)`.
Caused by warning:
! NAs introduced by coercion"


[1] 395  18

[1] 583   7

In [10]:
cleaned_data_act <- cleaned_data_act[, c( 'Sample_Code_16s', 'Sleep.Latency' , 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool')]

# Convert "nan" string to actual NA and ensure numeric conversion for specific columns
columns_to_clean <- c('Sleep.Latency', 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool')

cleaned_data_act[columns_to_clean] <- lapply(cleaned_data_act[columns_to_clean], function(x) {
  x[x == "nan"] <- NA  # Replace "nan" string with NA
  as.numeric(x)        # Convert the column to numeric
})




Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"
Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"


In [11]:

# Convert "nan" string to actual NA and ensure numeric conversion for specific columns
columns_to_clean <- c('Melatonin_pg_g_Stool')

data_samples[columns_to_clean] <- lapply(data_samples[columns_to_clean], function(x) {
  x[x == "nan"] <- NA  # Replace "nan" string with NA
  as.numeric(x)        # Convert the column to numeric
})




Warning message in FUN(X[[i]], ...):
"NAs introduced by coercion"


In [12]:
merged_data <- merge(data_samples,cleaned_data_act, by = "Sample_Code_16s")
dim(merged_data)
# Remove rows with NA or NaN
library(dplyr)

library(dplyr)

# Now remove rows with NA or NaN
merged_data <- merged_data %>%
  filter(complete.cases(.))

dim(merged_data)


[1] 395  12

[1] 358  12

In [13]:
#merged_data_age <- merged_data

if (Age == '3 months'){
    merged_data <- merged_data[merged_data$age_group ==3,]}

if (Age == '6 months'){
    merged_data <- merged_data[merged_data$age_group ==6,]}

if (Age == '12 months'){
    merged_data <- merged_data[merged_data$age_group ==12,]}

dim(merged_data)
merged_data[1:3,]

[1] 358  12

Sample_Code_16s Cohort Sample_id Timepoint Day Melatonin_pg_g_Stool age_group
1 S001-12         SDEGU  S001      t3        2   178.69               12       
2 S003-03         SDEGU  S003      t1        1   259.23                3       
3 S003-12         SDEGU  S003      t3        2    81.71               12       
  Sleep.Latency Sleep_duration_night Sleep_duration_day Awake_night Clock_stool
1 0.08          11.5                  2.0               0.33        14.30000   
2 0.25           9.0                  3.5               1.00        18.33333   
3 0.16          10.0                 35.0               0.25         8.50000

In [14]:
# Check for duplicates
if (any(duplicated(merged_data$Sample_Code_16s))) {
  # Remove duplicates, keeping only the first occurrence
  merged_data <- merged_data[!duplicated(merged_data$Sample_Code_16s), ]
  
  # Print a message indicating duplicates were removed
  message("Duplicates were found and removed.")
} else {
  # Print a message indicating no duplicates were found
  message("No duplicates were found.")
}

# View the cleaned dataframe
head(merged_data)
dim(merged_data)

Duplicates were found and removed.



Sample_Code_16s Cohort Sample_id Timepoint Day Melatonin_pg_g_Stool age_group
1 S001-12         SDEGU  S001      t3        2   178.69               12       
2 S003-03         SDEGU  S003      t1        1   259.23                3       
3 S003-12         SDEGU  S003      t3        2    81.71               12       
4 S004-06         SDEGU  S004      t2        2   125.28                6       
5 S004-12         SDEGU  S004      t3        2   116.47               12       
6 S005-06         SDEGU  S005      t2        1   157.80                6       
  Sleep.Latency Sleep_duration_night Sleep_duration_day Awake_night Clock_stool
1 0.08          11.5                  2.0               0.33        14.30000   
2 0.25           9.0                  3.5               1.00        18.33333   
3 0.16          10.0                 35.0               0.25         8.50000   
4 0.00          10.0                  3.0               0.75        11.75000   
5 0.16          10.0                  2.0               0.16        13.08333   
6 0.00          10.5                  3.0               0.50        11.00000

[1] 357  12

In [18]:
merged_data$baby_id <- sub("-.*", "", merged_data$Sample_Code_16s)
head(merged_data)

dim(merged_data)

Sample_Code_16s Cohort Sample_id Timepoint Day Melatonin_pg_g_Stool age_group
1 S001-12         SDEGU  S001      t3        2   178.69               12       
2 S003-03         SDEGU  S003      t1        1   259.23                3       
3 S003-12         SDEGU  S003      t3        2    81.71               12       
4 S004-06         SDEGU  S004      t2        2   125.28                6       
5 S004-12         SDEGU  S004      t3        2   116.47               12       
6 S005-06         SDEGU  S005      t2        1   157.80                6       
  Sleep.Latency Sleep_duration_night Sleep_duration_day Awake_night Clock_stool
1 0.08          11.5                  2.0               0.33        14.30000   
2 0.25           9.0                  3.5               1.00        18.33333   
3 0.16          10.0                 35.0               0.25         8.50000   
4 0.00          10.0                  3.0               0.75        11.75000   
5 0.16          10.0                  2.0               0.16        13.08333   
6 0.00          10.5                  3.0               0.50        11.00000   
  baby_id
1 S001   
2 S003   
3 S003   
4 S004   
5 S004   
6 S005

[1] 357  13

In [15]:
# Load necessary libraries
library(Maaslin2)
if (Age !='all'){
# Create input_data as a data frame and set row names
input_data1 <- merged_data[, c('Melatonin_pg_g_Stool'), drop = FALSE]
rownames(input_data1) <- merged_data$Sample_Code_16s

# Create input_metadata as a data frame and set row names
input_metadata1 <- merged_data[, c('Sleep.Latency', 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool'), drop = FALSE]
rownames(input_metadata1) <- merged_data$Sample_Code_16s

# Check and clean missing data
input_data1 <- na.omit(input_data1)
input_metadata1 <- na.omit(input_metadata1)

# Ensure output directory exists
dir.create("results", showWarnings = FALSE)

# Run MaAslin2 for multivariate testing
maaslin_results <- Maaslin2(
    input_data = input_data1,
    input_metadata = input_metadata1,
    output = paste0("results/", Age,"/"),
    fixed_effects = c('Sleep.Latency', 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool'), # Include independent variables and covariate
    random_effects = NULL, # Use random effects if needed
    normalization = 'NONE', # Or another method if required
    standardize = TRUE
)
    }


Warning message:
"package 'Maaslin2' was built under R version 4.4.2"


In [16]:
maaslin_results

ERROR: Error: object 'maaslin_results' not found


In [19]:
# Load necessary libraries
library(Maaslin2)
if(Age=='all'){
# Create input_data as a data frame and set row names
input_data1 <- merged_data[, c('Melatonin_pg_g_Stool'), drop = FALSE]
rownames(input_data1) <- merged_data$Sample_Code_16s

# Create input_metadata as a data frame and set row names
input_metadata1 <- merged_data[, c('Sleep.Latency', 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool', 'age_group', 'baby_id'), drop = FALSE]
rownames(input_metadata1) <- merged_data$Sample_Code_16s

# Check and clean missing data
input_data1 <- na.omit(input_data1)
input_metadata1 <- na.omit(input_metadata1)

# Ensure output directory exists
output_dir <- paste0("results/random_effects/", Age, "/", Study_variable, "/")
dir.create(output_dir, recursive = TRUE, showWarnings = FALSE)

# Run MaAslin2 for multivariate testing
maaslin_results <- Maaslin2(
    input_data = input_data1,
    input_metadata = input_metadata1,
    output = output_dir,
    fixed_effects = c('Sleep.Latency', 'Sleep_duration_night', 'Sleep_duration_day', 'Awake_night', 'Clock_stool'), # Include independent variables and covariate
    random_effects = c('baby_id'), # ✅ Fixed typo
    normalization = 'NONE', # Or another method if required
    standardize = TRUE
)
    }


[1] "Creating output feature tables folder"
[1] "Creating output fits folder"
[1] "Creating output figures folder"
2025-04-02 18:13:04.660103 INFO::Writing function arguments to log file
2025-04-02 18:13:04.681856 INFO::Verifying options selected are valid
2025-04-02 18:13:04.707113 INFO::Determining format of input files
2025-04-02 18:13:04.708509 INFO::Input format is data samples as rows and metadata samples as rows
2025-04-02 18:13:04.711075 INFO::Formula for random effects: expr ~ (1 | baby_id)
2025-04-02 18:13:04.712324 INFO::Formula for fixed effects: expr ~  Sleep.Latency + Sleep_duration_night + Sleep_duration_day + Awake_night + Clock_stool
2025-04-02 18:13:04.713384 INFO::Filter data based on min abundance and min prevalence
2025-04-02 18:13:04.714308 INFO::Total samples in data: 357
2025-04-02 18:13:04.715234 INFO::Min samples required with min abundance for a feature not to be filtered: 35.700000
2025-04-02 18:13:04.716541 INFO::Total filtered features: 0
2025-04-02 18:13:

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


2025-04-02 18:13:06.507561 INFO::Plotting data for metadata number 2, Sleep.Latency
2025-04-02 18:13:06.509303 INFO::Creating scatter plot for continuous data, Sleep.Latency vs Melatonin_pg_g_Stool


`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In [20]:
maaslin_results

$results
               feature             metadata                value        coef
1 Melatonin_pg_g_Stool          Clock_stool          Clock_stool -0.19776320
2 Melatonin_pg_g_Stool        Sleep.Latency        Sleep.Latency  0.07950351
3 Melatonin_pg_g_Stool Sleep_duration_night Sleep_duration_night -0.02513919
4 Melatonin_pg_g_Stool   Sleep_duration_day   Sleep_duration_day -0.01553633
5 Melatonin_pg_g_Stool          Awake_night          Awake_night  0.02309256
      stderr         pval                 name         qval   N N.not.zero
1 0.04536121 1.731474e-05          Clock_stool 8.657369e-05 357        357
2 0.04468164 7.616444e-02        Sleep.Latency 1.904111e-01 357        357
3 0.05049944 6.189293e-01 Sleep_duration_night 7.279195e-01 357        357
4 0.04461820 7.279195e-01   Sleep_duration_day 7.279195e-01 357        357
5 0.05014363 6.454227e-01          Awake_night 7.279195e-01 357        357

$residuals
                       S001-12  S003-03   S003-12    S004-06    S004-12
Melatonin_pg_g_Stool 0.2995434 1.028463 -1.009145 -0.1364685 -0.1696935
                       S005-06    S005-12   S006-03    S006-06   S006-12
Melatonin_pg_g_Stool 0.0442988 0.01563689 0.3475949 -0.8894475 0.7609827
                        S007-03   S007-06      S007-12    S009-06    S009-12
Melatonin_pg_g_Stool -0.9455017 0.5298752 -0.008360842 -0.3535157 -0.2967088
                        S010-12   S011-06    S011-12   S012-06   S012-12
Melatonin_pg_g_Stool -0.4792256 -1.908281 -0.2979594 0.2046272 0.1157439
                       S013-03  S013-12    S014-03   S014-06    S014-12
Melatonin_pg_g_Stool -0.184807 0.956579 -0.1783215 -1.254507 -0.3374035
                        S015-03   S015-12    S016-03   S016-06   S019-12
Melatonin_pg_g_Stool -0.7649208 0.6364357 -0.6494689 0.1104796 0.1553556
                        S020-03     S020-06   S020-12  S021-03    S021-06
Melatonin_pg_g_Stool -0.6202905 -0.00350241 0.9766012 0.239281 -0.3784255
                       S021-12    S022-03     S022-06    S023-03   S023-06
Melatonin_pg_g_Stool 0.4628165 -0.2970772 -0.08111198 -0.2331462 0.3422508
                       S023-12    S024-06  S025-03   S025-12    S026-03
Melatonin_pg_g_Stool 0.0418322 0.08901231 1.552075 -1.002539 -0.2883425
                        S026-06   S026-12    S027-06   S027-12    S028-06
Melatonin_pg_g_Stool 0.05988847 -0.418445 -0.9434493 0.4222598 -0.4962214
                      S029-06     S029-12     S030-06    S030-12    S031-12
Melatonin_pg_g_Stool 1.392293 -0.04656657 -0.04642841 -0.2933393 -0.4543605
                        S032-03   S032-06   S033-06    S033-12   S034-03
Melatonin_pg_g_Stool -0.6765136 0.7226052 -0.250209 0.03455972 0.2465275
                         S034-12    S035-06    S035-12  S036-06    S037-03
Melatonin_pg_g_Stool -0.03659145 -0.7360698 -0.7109695 0.304519 -0.2018715
                       S037-06   S037-12   S038-03   S038-06  S038-12   S040-03
Melatonin_pg_g_Stool 0.8058606 0.7329458 -1.439409 0.3938885 1.095233 0.7049501
                        S040-12    S041-03   S041-06   S041-12    S042-06
Melatonin_pg_g_Stool -0.2273823 0.05758692 0.2375398 0.4848204 0.09018102
                      S042-12    S043-03   S043-06     S043-12     S044-03
Melatonin_pg_g_Stool 1.415008 -0.0462771 0.6697208 -0.04152097 -0.03000789
                        S044-06  S044-12  S045-12  S046-03  S046-06   S046-12
Melatonin_pg_g_Stool -0.9793459 1.519155 1.243653 0.643663 1.015563 -0.296722
                      S047-12   S048-03    S048-06   S048-12    S049-03
Melatonin_pg_g_Stool -0.71683 0.8228974 -0.2133879 0.2456596 -0.1011068
                       S049-06    S049-12    S050-03   S050-12   S051-03
Melatonin_pg_g_Stool 0.6560408 -0.7513161 -0.3680548 -0.337286 0.5866935
                       S051-06   S051-12    S052-03  S052-06   S052-12
Melatonin_pg_g_Stool -1.307374 0.1476461 -0.7481594 1.375845 0.3998811
                        S053-03   S053-06   S053-12    S054-03   S054-06
Melatonin_pg_g_Stool -0.298303